In this notebook, we try to find a solution to the set covering problem

Given a set of elements called the universe $U$ and a collection of subets of $U$ that we call $S = \{s_1, ..., s_n\} \subseteq 2^U$, such that:
\begin{equation*}
\bigcup_{i=1}^m s_i = U
\end{equation*}
The set covering problem consists on finding the collection $C^*$ composed by the minimum elements of $S$ such that it covers all the elements of $U$.

Here we model the set covering problem as a search problem, in particular we a generic state by $\sigma$, this state is characterised by a collection of subsets $C_\sigma \subseteq S$. In the following snippet of code we implement states using a Python class.

In [97]:
from search import BaseState
from typing import Optional, Hashable

class State(BaseState):

    @staticmethod
    def get_start_state(universe: set[Hashable], subsets: list[set[Hashable]]) -> 'State':
        collection: tuple[bool] = tuple([False for i in range(len(subsets))])
        return State(universe, subsets, collection, 0)

    def __init__(self, universe: set[Hashable], subsets: list[set[Hashable]], collection: tuple[bool], priority: float = 0, parent: Optional['State'] = None, level: int = 0) -> None:
        self.parent: Optional['State'] = parent
        self.subsets: list[set[Hashable]] = subsets
        self.universe: set[Hashable] = universe
        self.level: int = level
        super().__init__(collection, priority)
    
    def __repr__(self) -> str:
        ret: list[set[int]] = []
        for i, el in enumerate(self.collection):
            if el:
                ret.append(self.subsets[i])
        return str(ret)
    
    @property
    def complete_path(self) -> str:
        tmp: Optional[State] = self
        path: str = ''
        i: int = 0
        while tmp is not None:
            if i == 0:
                path = str(tmp)
                i = 1
            else:
                path = path + ' <---- ' + str(tmp)
            tmp = tmp.parent
        return path

    
    @property
    def collection(self):
        return self.state_identifier
    
    @collection.setter
    def collection(self, collection):
        self.state_identifier = collection
    
    def __len__(self) -> int:
        try:
            return len([i for i in self.collection if i])
        except:
            raise TypeError(f'Collection not valid')
    
    def explore(self) -> list['State']:
        children: list['State'] = []
        new_collection: list[bool] = list(self.collection)
        for i in range(self.level, len(self.collection)):
            if not new_collection[i]:
                new_collection[i] = True
                children.append(State(self.universe, self.subsets, tuple(new_collection), parent=self, level=self.level+1))
                new_collection[i] = False
        return children

In order to understand whether a state is better than another we simply count the number of subsets of the state.

In [98]:
def f_evaluate(s1: State, s2: State) -> bool:

    length_collection1 : int = len([a for a in s1.collection if a])
    length_collection2 : int = len([a for a in s2.collection if a])

    return length_collection1 >= length_collection2

Likewise, we need also to understand whether a state solves the problem. This is done by computing the union of the subsets of the state and checking whether the union coincides with the universe or not.

In [99]:
def f_goal(s: State):
    union = set()
    for i, subset in enumerate(s.subsets):
        if s.collection[i]:
            union = union | subset
    return union == s.universe

It is interesting to see that graph search takes significantly less to find the best solution compared to tree search.

In [100]:
from search import TreeSearch
from search import GraphSearch
import time
import numpy as np

ts = TreeSearch()
gs = GraphSearch()

universe = set(range(12))
subsets = [set(np.random.randint(0,12, size=i)) for i in range(3,7) for j in range(2)]

t1 = time.time()
ts.best_search(State.get_start_state(universe, subsets), f_evaluate, f_goal)
t2 = time.time()
print(f'Time for tree search {t2-t1}')
print(f'Number of steps: {ts.num_steps}')

t1 = time.time()
gs.best_search(State.get_start_state(universe, subsets), f_evaluate, f_goal)
t2 = time.time()
print(f'Time for graph search {t2-t1}')
print(f'Number of steps: {gs.num_steps}')

Doggos
Time for tree search 0.06690764427185059
Number of steps: 2781
Time for graph search 0.10304522514343262
Number of steps: 256


We can also apply the $A^*$ algorithm. We can do that exploiting the priority function $f_p$, which defines the priority key in the priority queue of the algorthm. In particular the $A^*$ algorithm requires $f_p$ to be of the form:
\begin{equation}
f_p = f_c + h
\end{equation}
Where $f_c$ is the cost function, whereas $h$ is the heuristic. In our case, the cost function is defined by the cardinality of the collection of sets $C_\sigma$ of state $\sigma$:
\begin{equation}
f_c(\sigma) = |C_\sigma|
\end{equation}

In [101]:
def f_cost(s: State):
    return sum(s.collection)

We also have to define a heuristic function $h_1$, the function $h_1$, to satisfy the requirements for $A^*$ to be complete and optimal, must be admissible, that is it should never overestimate the distance of a state from the actual best solution. 

We know that the best solution in set covering has the minimum number of sets, hence, if we ignore the elements inside the remaining sets, then we can infer that the number of additional sets that is needed to get the optimal solution is larger or equal than the number of sets such that the sum of the cardinalities of those sets is equal or larger to the cardinality of the universe.

Therefore we can define the heuristic $h_1$ as follows:
\begin{equation}
h_1(\sigma) = \underset{n}{\text{min}}\{n \in \mathbb{N}|\sum_{\alpha \in C_\sigma}|\alpha| + \sum_{i=1}^n |{L_\sigma}_i| \geq |U|\}
\end{equation}
Where $L_\sigma = S\setminus C_\sigma$ is an ordered list of the subsets in $S$ which are not included in $L_\sigma$, in particular the sets in $L_\sigma$ are arranged in decreasing order of cardinality.

In [102]:
def f_heuristic1(s: State):
    collection_cardinality: int = 0
    remaining_subset_lengths: list[int] = []

    for i, el in enumerate(s.collection):
        if not el:
            remaining_subset_lengths.append(len(s.subsets[i]))
        else:
            collection_cardinality += len(s.subsets[i])
    
    remaining_subset_lengths = sorted(remaining_subset_lengths, reverse=True)
    
    n: int = 0
    cumulative_cardinality: int = 0

    while collection_cardinality + cumulative_cardinality < len(s.universe):

        if n>=len(remaining_subset_lengths):
            break

        cumulative_cardinality += remaining_subset_lengths[n]
        n += 1
    
    return n

Thus, we can define our priority function as:

In [103]:
def f_priority(s: State,):
    return f_cost(s) + f_heuristic1(s)

We have all the elements to solve the problem with $A^*$:

In [104]:
from search import TreeSearch
from search import GraphSearch
from time import time

ts: TreeSearch = TreeSearch()
gs: GraphSearch = GraphSearch()
PROBLEM_SIZE: int = 4
MIN_SET_SIZE: int = 1
MAX_SET_SIZE: int = 8
MINIMUM_NUMBER_OF_SUBSETS: int = 3
universe: set[int] = set(range(PROBLEM_SIZE))
subsets: list[set[int]] = []
i: int = 0
universe_covered: bool = False
universe_coverage: set[int] = set()

while i < MINIMUM_NUMBER_OF_SUBSETS or not universe_covered:
    set_size = np.random.randint(MIN_SET_SIZE,MAX_SET_SIZE)
    set_ = set(np.random.randint(0,PROBLEM_SIZE, size=set_size))
    universe_coverage |= set_
    universe_covered = universe_coverage == universe
    subsets.append(set_)
    i+=1

print(f'Actual size {i}')
print(f'Universe {universe}')
print(f'Subsets {subsets}')

ts = TreeSearch()
t1: float = time()
ts.objective_search(State.get_start_state(universe, subsets), f_goal, f_priority)
t2: float = time()
print(f'Time with heuristic {t2-t1}')
print(f'Number of steps: {ts.num_steps}')
print(f'Solution {ts.solution}, cardinality {len(ts.solution)}')
print(f'Solution path {ts.solution.complete_path}')
print(f'Solution cost {ts.solution.priority}')

ts = TreeSearch()
gs = GraphSearch()
t1: float = time()
gs.best_search(State.get_start_state(universe, subsets), f_evaluate, f_goal)
t2: float = time()
print(f'Time without {t2-t1}')
print(f'Number of steps: {gs.num_steps}')
print(f'Solution {gs.solution}, cardinality {len(gs.solution)}')
print(f'Solution path {gs.solution.complete_path}')
print(f'Solution cost {gs.solution.priority}')

Doggos
Actual size 3
Universe {0, 1, 2, 3}
Subsets [{1, 3}, {0, 1, 2, 3}, {0, 1, 3}]
Doggos
Time with heuristic 0.0
Number of steps: 2
Solution [{0, 1, 2, 3}], cardinality 1
Solution path [{0, 1, 2, 3}] <---- []
Solution cost 1.0
Doggos
Time without 0.0
Number of steps: 8
Solution [{0, 1, 2, 3}], cardinality 1
Solution path [{0, 1, 2, 3}] <---- []
Solution cost 3.0


Is $h_1$ also consistent? We can prove that. Suppose that the states $\sigma_1$ and $\sigma_2$ are connected by a (directed) edge, then we test:
\begin{equation*}
h_1(\sigma_1) \leq d(\sigma_1,\sigma_2) + h_1(\sigma_2)
\end{equation*}
Since in our setting $d(\sigma_1,\sigma_2) = 1$ we write:
\begin{equation*}
h_1(\sigma_1) \leq 1 + h_1(\sigma_2)
\end{equation*}
We can observe that $C_{\sigma_2} = C_{\sigma_1}\setminus s$, where $s$ is the extra subset that $\sigma_2$ took in its transition from its parent $\sigma_1$. To avoid a cluttered notation we use directly $i$ instead of $\sigma_i$ in subscripts. We can infer, then, the following:
\begin{equation*}
L_2 = L_1 \setminus \{s\}
\end{equation*}
Now, we have two options:
1. $s = {L_1}_1 \implies \forall i \in [0,n-1]\ |{L_1}_1| + \sum_{j=2}^{i+1} |{L_1}_{j}| = |s| + \sum_{j=1}^{i} |{L_2}_j| \implies h_1(\sigma_1) = h_1(\sigma_2) + 1$ 
2. $s \neq {L_1}_1 \implies \exists j | \forall i \in [0, j]\ |{L_1}_1| + \sum_{k=2}^{i+1} |{L_1}_{k}| \geq |s| + \sum_{k=1}^{i} |{L_2}_k| \implies h_1(\sigma_1) \leq h_1(\sigma_2) + 1$

Overall, we get $h_1(\sigma_1) \leq h_1(\sigma_2) + 1$, which is the definition of admissibility. As an implication, we can make use of $h_1$ in the graph search version of $A^*$, guaranteeing optimality.

We can also define a new heuristic $h_2$, starting from the previous one. The new heuristic works similarly to the previous one. However, instead of using the cardinality of the remaining subsets, we use the cardinality of the complement of the intersection of the remaining subsets with the part of universe covered by the current set. More formally:
\begin{equation*}
    h_2(\sigma) = \underset{n}{\text{min}}\{n \in \mathbb{N}|\sum_{\alpha \in C_\sigma}|\alpha| + \sum_{i=1}^n |{L_\sigma^*}_i| \geq |U|\}
\end{equation*}
In particular, we construct $L_\sigma^*$ as follows:
1. We compute the universe coverage $U_\sigma = \bigcup_{\alpha \in C_\sigma} \alpha$.
2. For each of the remaining sets, we remove the elements in $U_\sigma$ by taking the complement of the intersection, that is, for $s$ in $S\setminus C_\sigma$ we have $s^* = {U_\sigma \cup s}^c$.
3. We put all the $s^*$ in $L_\sigma^*$, arranged by decreasing order of cardinality.


In [105]:
def f_heuristic2(s: State):
    collection_cardinality: int = 0
    remaining_subset_lengths: list[int] = []

    universe_coverage: set[int] = set()

    for i, el in enumerate(s.collection):
        if el:
            universe_coverage |= s.subsets[i]

    for i, el in enumerate(s.collection):
        if not el:
            remaining_subset_lengths.append(len(s.subsets[i] & universe_coverage))
        else:
            collection_cardinality += len(s.subsets[i])
    
    remaining_subset_lengths = sorted(remaining_subset_lengths, reverse=True)
    
    n: int = 0
    cumulative_cardinality: int = 0

    while collection_cardinality + cumulative_cardinality < len(s.universe):

        if n>=len(remaining_subset_lengths):
            break

        cumulative_cardinality += remaining_subset_lengths[n]
        n += 1
    
    return n

We can again make some tests

In [106]:
from search import TreeSearch
from search import GraphSearch
from time import time

ts: TreeSearch = TreeSearch()
gs: GraphSearch = GraphSearch()
PROBLEM_SIZE: int = 15
MIN_SET_SIZE: int = 1
MAX_SET_SIZE: int = 8
MINIMUM_NUMBER_OF_SUBSETS: int = 3
universe: set[int] = set(range(PROBLEM_SIZE))
subsets: list[set[int]] = []
i: int = 0
universe_covered: bool = False
universe_coverage: set[int] = set()

while i < MINIMUM_NUMBER_OF_SUBSETS or not universe_covered:
    set_size = np.random.randint(MIN_SET_SIZE,MAX_SET_SIZE)
    set_ = set(np.random.randint(0,PROBLEM_SIZE, size=set_size))
    universe_coverage |= set_
    universe_covered = universe_coverage == universe
    subsets.append(set_)
    i+=1

print(f'Actual size {i}')
print(f'Universe {universe}')
print(f'Subsets {subsets}')

f_p1 = lambda x: f_cost(x) + f_heuristic1(x)
f_p2 = lambda x: f_cost(x) + f_heuristic2(x)

ts = TreeSearch()
t1: float = time()
ts.objective_search(State.get_start_state(universe, subsets), f_goal, f_p1)
t2: float = time()
print(f'Time with heuristic 1:{t2-t1}')
print(f'Number of steps: {ts.num_steps}')
print(f'Solution {ts.solution}, cardinality {len(ts.solution)}')
print(f'Solution path {ts.solution.complete_path}')
print(f'Solution cost {ts.solution.priority}')

ts = TreeSearch()
t1: float = time()
ts.objective_search(State.get_start_state(universe, subsets), f_goal, f_p2)
t2: float = time()
print(f'Time with heuristic 2:{t2-t1}')
print(f'Number of steps: {ts.num_steps}')
print(f'Solution {ts.solution}, cardinality {len(ts.solution)}')
print(f'Solution path {ts.solution.complete_path}')
print(f'Solution cost {ts.solution.priority}')

Doggos
Actual size 8
Universe {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
Subsets [{0, 5, 6, 7, 14}, {1, 11, 13}, {8, 4, 12, 6}, {2, 4, 6, 7, 10}, {10, 11, 4, 6}, {6}, {2, 3, 5, 6, 11, 13}, {0, 3, 5, 9, 10, 13}]
Doggos
Time with heuristic 1:0.04799842834472656
Number of steps: 1178
Solution [{0, 5, 6, 7, 14}, {1, 11, 13}, {8, 4, 12, 6}, {2, 3, 5, 6, 11, 13}, {0, 3, 5, 9, 10, 13}], cardinality 5
Solution path [{0, 5, 6, 7, 14}, {1, 11, 13}, {8, 4, 12, 6}, {2, 3, 5, 6, 11, 13}, {0, 3, 5, 9, 10, 13}] <---- [{0, 5, 6, 7, 14}, {1, 11, 13}, {8, 4, 12, 6}, {0, 3, 5, 9, 10, 13}] <---- [{0, 5, 6, 7, 14}, {1, 11, 13}, {8, 4, 12, 6}] <---- [{0, 5, 6, 7, 14}, {1, 11, 13}] <---- [{0, 5, 6, 7, 14}] <---- []
Solution cost 5.0
Doggos
Time with heuristic 2:0.024972200393676758
Number of steps: 632
Solution [{0, 5, 6, 7, 14}, {1, 11, 13}, {8, 4, 12, 6}, {2, 4, 6, 7, 10}, {0, 3, 5, 9, 10, 13}], cardinality 5
Solution path [{0, 5, 6, 7, 14}, {1, 11, 13}, {8, 4, 12, 6}, {2, 4, 6, 7, 10}, {0, 3, 5, 9

As we can see this new heuristic performs considerably better, this is due to the fact that it gives a closer estimate of the actual distance, while also keeping the admissibility property. However notice that it is more expensive to compute, since we have additional intersections.

An even better version consists on iterating such a procedure. We can find the best set for the state with the criteria of $h_2$, then we insert it in the state and we repeat the procedure. In this case the distance becomes the number of iterations necessary to get to a solution of the set covering problem.

In [107]:
def calculate_universe_coverage(collection: list[bool], subsets: list[set[int]]):
    universe_coverage: set[int] = set()

    for i, el in enumerate(collection):
        if el:
            universe_coverage |= subsets[i]
    return universe_coverage

def f_heuristic3(s: State):
    
    universe_coverage: set[int] = calculate_universe_coverage(s.collection, s.subsets)
    tmp_collection: list[bool] = list(s.collection)

    n: int = 0

    while not universe_coverage == s.universe:
        max_size: int = -1
        max_subset_index: int = -1
        for i, el in enumerate(tmp_collection):
            if not el:
                candidate_subset: set[int] = s.subsets[i] | universe_coverage
                if len(candidate_subset) > max_size:
                    max_subset_index = i
                    max_size = len(candidate_subset)
        tmp_collection[max_subset_index] = True
        universe_coverage = calculate_universe_coverage(tmp_collection, s.subsets)
        n += 1
    
    return n

Again, we can perform some experiments:

In [108]:
from search import TreeSearch
from search import GraphSearch
from time import time

ts: TreeSearch = TreeSearch()
gs: GraphSearch = GraphSearch()
PROBLEM_SIZE: int = 15
MIN_SET_SIZE: int = 1
MAX_SET_SIZE: int = 8
MINIMUM_NUMBER_OF_SUBSETS: int = 3
universe: set[int] = set(range(PROBLEM_SIZE))
subsets: list[set[int]] = []
i: int = 0
universe_covered: bool = False
universe_coverage: set[int] = set()

while i < MINIMUM_NUMBER_OF_SUBSETS or not universe_covered:
    set_size = np.random.randint(MIN_SET_SIZE,MAX_SET_SIZE)
    set_ = set(np.random.randint(0,PROBLEM_SIZE, size=set_size))
    universe_coverage |= set_
    universe_covered = universe_coverage == universe
    subsets.append(set_)
    i+=1

subsets = [{11, 4, 13, 6}, {3, 14}, {1, 4, 7, 10, 11, 14}, {14, 6}, {0, 12, 14}, {5}, {1, 2, 8, 11, 12, 13, 14}, {0, 5, 7, 8, 9}]

print(f'Actual size {i}')
print(f'Universe {universe}')
print(f'Subsets {subsets}')

f_p1 = lambda x: f_cost(x) + f_heuristic1(x)
f_p2 = lambda x: f_cost(x) + f_heuristic2(x)
f_p3 = lambda x: f_cost(x) + f_heuristic3(x)

ts = TreeSearch()
t1: float = time()
ts.objective_search(State.get_start_state(universe, subsets), f_goal, f_p1)
t2: float = time()
print(f'Time with heuristic 1:{t2-t1}')
print(f'Number of steps: {ts.num_steps}')
print(f'Solution {ts.solution}, cardinality {len(ts.solution)}')
print(f'Solution cost {ts.solution.priority}')

ts = TreeSearch()
t1: float = time()
ts.objective_search(State.get_start_state(universe, subsets), f_goal, f_p2)
t2: float = time()
print(f'Time with heuristic 2:{t2-t1}')
print(f'Number of steps: {ts.num_steps}')
print(f'Solution {ts.solution}, cardinality {len(ts.solution)}')
print(f'Solution cost {ts.solution.priority}')

ts = TreeSearch()
t1: float = time()
ts.objective_search(State.get_start_state(universe, subsets), f_goal, f_p3)
t2: float = time()
print(f'Time with heuristic 3:{t2-t1}')
print(f'Number of steps: {ts.num_steps}')
print(f'Solution {ts.solution}, cardinality {len(ts.solution)}')
print(f'Solution cost {ts.solution.priority}')
print(f'Solution path {ts.solution.complete_path}')

Doggos
Actual size 10
Universe {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
Subsets [{11, 4, 13, 6}, {3, 14}, {1, 4, 7, 10, 11, 14}, {14, 6}, {0, 12, 14}, {5}, {1, 2, 8, 11, 12, 13, 14}, {0, 5, 7, 8, 9}]
Doggos
Time with heuristic 1:0.03200030326843262
Number of steps: 634
Solution [{3, 14}, {1, 4, 7, 10, 11, 14}, {14, 6}, {1, 2, 8, 11, 12, 13, 14}, {0, 5, 7, 8, 9}], cardinality 5
Solution cost 5.0
Doggos
Time with heuristic 2:0.0050008296966552734
Number of steps: 147
Solution [{3, 14}, {1, 4, 7, 10, 11, 14}, {14, 6}, {1, 2, 8, 11, 12, 13, 14}, {0, 5, 7, 8, 9}], cardinality 5
Solution cost 5.0
Doggos
Time with heuristic 3:0.003000020980834961
Number of steps: 40
Solution [{3, 14}, {1, 4, 7, 10, 11, 14}, {14, 6}, {1, 2, 8, 11, 12, 13, 14}, {0, 5, 7, 8, 9}], cardinality 5
Solution cost 5.0
Solution path [{3, 14}, {1, 4, 7, 10, 11, 14}, {14, 6}, {1, 2, 8, 11, 12, 13, 14}, {0, 5, 7, 8, 9}] <---- [{3, 14}, {1, 4, 7, 10, 11, 14}, {14, 6}, {1, 2, 8, 11, 12, 13, 14}] <---- [{3, 14}, {1,

We have to denote that $h_3$ is not admissible, that is because every time it goes on to compute a valid solution, which is not the best one.
More precisely, the point before provides the argument for the heuristic computing a cost which is larger or _equal_ to an optimal solution.
However, we can try to make heuristic by bounding the optimal solution. Trivially, we can bound the distance of the optimal solution with the one computed by the heuristic. However, we also know that, by construction, $h_3$ can provide an optimal solution. What about two steps away?
Is there a case where $h_3$ could take three steps or more? We have to consider that, over two steps, another choice could be better. That is because the best solution over two steps have to consider all possible combinations of two choices.

We can make an example:
$U = \{1,2,3,4,5,6\}$
$S = \{\{1,2,3,4\}, \{2,4,6\}, \{1,3,5\}\}$.
In this case the optimal solution is $\{\{2,4,6\}, \{1,3,4,5\}\}$; $h_3$, however would take $\{1,2,3,4\}$ first, which would force taking the other two sets. A similar argument goes for the heuristic $h_2$, since it would follow the same path as $h_3$ in this case.

In [109]:
from search import TreeSearch

ts: TreeSearch = TreeSearch()

subsets: list[set[int]] = [{1,2,3,4}, {2,4,6}, {1,3,5}]
universe: set[int] = {1,2,3,4,5,6}

f_p1 = lambda x: f_cost(x) + f_heuristic1(x)

initial_state: State = State.get_start_state(universe, subsets)
ts.objective_search(initial_state, f_goal=f_goal, f_priority= lambda x: f_p1(x))
print(f'Actual solution: {ts.solution.complete_path}')
print(f'Cost h1: {f_heuristic1(initial_state)}')
print(f'Cost h2: {f_heuristic2(initial_state)}')
print(f'Cost h3: {f_heuristic3(initial_state)}')


Doggos
Actual solution: [{2, 4, 6}, {1, 3, 5}] <---- [{2, 4, 6}] <---- []
Cost h1: 2
Cost h2: 3
Cost h3: 3


However, note that we can make both $h_3$ and $h_2$ admissible by dividing by a constant $k$. Moreover, we can make use of $h_1$, since it is admissible. In particular, if $\sigma$ is a state and $f(\sigma)$ is the exact distance of $\sigma$ from the solution, we can find $k$ by solving the system:
$$\begin{cases}
h_1(\sigma) &\leq f(s) \\
\frac{h_3(\sigma)}{k} &\leq f(s)
\end{cases}$$
In particular we use $\frac{h_3(\sigma)}{k} \leq h_1(\sigma) \implies \frac{h_3(\sigma)}{k} \leq f(\sigma)$. Then we get $k \geq \frac{h_3(\sigma)}{h_1(\sigma)}$. Note, however that this does not make sense, since at this point we could just use $h_1$. In general we shall find a bound that is not computed by a heuristic, but that's not possible, since a heuristic is any function of the state. The strength of $h_3$ lies in the fact that it can find a very good solution in a much shorter time than $h_1$, at least from what it's shown by the tests performed in this notebook. This can be enough for many application which do not require an exact solution. 

In [110]:
from search import TreeSearch

ts: TreeSearch = TreeSearch()

subsets: list[set[int]] = [{1,2,3,4}, {2,4,6}, {1,3,5}]
universe: set[int] = {1,2,3,4,5,6}

f_p1 = lambda x: f_cost(x) + f_heuristic1(x)
f_h3_2 = lambda x: f_heuristic3(x)/(f_heuristic3(x)/f_heuristic1(x)) # We calculate k=f_h3/f_h1, but notice that if we simplify it is just f_h1!

initial_state: State = State.get_start_state(universe, subsets)
ts.objective_search(initial_state, f_goal=f_goal, f_priority= f_p1)
print(f'Actual solution: {ts.solution.complete_path}')
print(f'Cost h1: {f_heuristic1(initial_state)}')
print(f'Cost normalised h3: {f_h3_2(initial_state)}')

Doggos
Actual solution: [{2, 4, 6}, {1, 3, 5}] <---- [{2, 4, 6}] <---- []
Cost h1: 2
Cost normalised h3: 2.0
